In [ ]:
#initialize
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms 
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
 
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

 #hyperparameters data size 28*28*1, 10 classes
num_classes = 10
in_channel = 1
learning_rate = 0.001
batch_size = 128
num_epochs = 30

In [ ]:
from google.colab import drive 

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
csv_dataset1_train_alice1 ='/content/gdrive/MyDrive/dataset1/AIcomm_project_2_train_alice_1.csv'
csv_dataset1_train_john1 ='/content/gdrive/MyDrive/dataset1/AIcomm_project_2_train_john_1.csv'
csv_dataset1_train_peter1 ='/content/gdrive/MyDrive/dataset1/AIcomm_project_2_train_peter_1.csv'
csv_dataset2_train_alice2 ='/content/gdrive/MyDrive/dataset2/AIcomm_project_2_train_alice_2.csv'
csv_dataset2_train_john2 ='/content/gdrive/MyDrive/dataset2/AIcomm_project_2_train_john_2.csv'
csv_dataset2_train_peter2 ='/content/gdrive/MyDrive/dataset2/AIcomm_project_2_train_peter_2.csv'

cols_dataset1_train_alice1 = list(pd.read_csv(csv_dataset1_train_alice1, nrows =1))
cols_dataset1_train_john1 = list(pd.read_csv(csv_dataset1_train_john1, nrows =1))
cols_dataset1_train_peter1 = list(pd.read_csv(csv_dataset1_train_peter1, nrows =1))
cols_dataset2_train_alice2 = list(pd.read_csv(csv_dataset2_train_alice2, nrows =1))
cols_dataset2_train_john2 = list(pd.read_csv(csv_dataset2_train_john2, nrows =1))
cols_dataset2_train_peter2 = list(pd.read_csv(csv_dataset2_train_peter2, nrows =1))



csv_features_alice1 = pd.read_csv(csv_dataset1_train_alice1, index_col = 0, usecols =[i for i in cols_dataset1_train_alice1 if i != 'label'])
csv_label_alice1 = pd.read_csv(csv_dataset1_train_alice1, index_col = 0, usecols=[0, 1])
csv_features_john1 = pd.read_csv(csv_dataset1_train_john1, index_col = 0, usecols =[i for i in cols_dataset1_train_john1 if i != 'label'])
csv_label_john1 = pd.read_csv(csv_dataset1_train_john1, index_col = 0, usecols=[0, 1])
csv_features_peter1 = pd.read_csv(csv_dataset1_train_peter1, index_col = 0, usecols =[i for i in cols_dataset1_train_peter1 if i != 'label'])
csv_label_peter1 = pd.read_csv(csv_dataset1_train_peter1, index_col = 0, usecols=[0, 1])

csv_features_alice2 = pd.read_csv(csv_dataset2_train_alice2, index_col = 0, usecols =[i for i in cols_dataset2_train_alice2 if i != 'label'])
csv_label_alice2 = pd.read_csv(csv_dataset2_train_alice2, index_col = 0, usecols=[0, 1])
csv_features_john2 = pd.read_csv(csv_dataset2_train_john2, index_col = 0, usecols =[i for i in cols_dataset2_train_john2 if i != 'label'])
csv_label_john2 = pd.read_csv(csv_dataset2_train_john2, index_col = 0, usecols=[0, 1])
csv_features_peter2 = pd.read_csv(csv_dataset2_train_peter2, index_col = 0, usecols =[i for i in cols_dataset2_train_peter2 if i != 'label'])
csv_label_peter2 = pd.read_csv(csv_dataset2_train_peter2, index_col = 0, usecols=[0, 1])

In [ ]:
count_class = []
labels = [csv_label_alice1,csv_label_john1,csv_label_peter1,csv_label_alice2,csv_label_john2,csv_label_peter2]
name_labels = ["csv_label_alice1","csv_label_john1","csv_label_peter1","csv_label_alice2","csv_label_john2","csv_label_peter2"]
counter = 0;

for i in range(10):
    count_class.append(0)

for lab in labels:
    for i in range(lab.shape[0]):
        label = lab.values[i][0]
        count_class[label] = count_class[label] + 1
    print(name_labels[counter], " :  ", count_class," \n")
    count_class = []
    for i in range(10):
        count_class.append(0)
    counter = counter +1

csv_label_alice1  :   [2002, 1947, 2026, 2048, 1999, 1965, 2051, 1980, 1927, 2055]  

csv_label_john1  :   [1952, 2053, 2046, 1983, 1988, 2003, 2017, 1966, 2046, 1946]  

csv_label_peter1  :   [2008, 2034, 1971, 1977, 2048, 2035, 1947, 2020, 1943, 2017]  

csv_label_alice2  :   [5978, 5985, 6023, 0, 0, 0, 0, 0, 0, 2014]  

csv_label_john2  :   [0, 0, 0, 0, 0, 0, 5963, 5961, 6065, 2011]  

csv_label_peter2  :   [0, 0, 0, 6067, 5979, 5983, 0, 0, 0, 1971]  



In [ ]:
csv_data ='/content/gdrive/MyDrive/AIcomm_project_2_test.csv'

cols = list(pd.read_csv(csv_data, nrows =1))

csv_features = pd.read_csv(csv_data, index_col = 0, usecols =[i for i in cols if i != 'label'])
csv_label = pd.read_csv(csv_data, index_col = 0, usecols=[0, 1])

count_class = []

for i in range(num_classes):
    count_class.append(0)

for i in range(csv_label.shape[0]):
    label = csv_label.values[i][0]
    count_class[label] = count_class[label] + 1

print("csv_test_label :  ", count_class," \n")

csv_test_label :   [1038, 966, 957, 992, 965, 997, 985, 1034, 1084, 982]  

